In [1]:
import os
print(os.getcwd()) #print working dir
import matplotlib.pyplot as plt
from matplotlib import gridspec
import time

os.sys.path.insert(0, '../src/')
from analyze_utilities import *
%run ./result_summary_uriya.ipynb

output_path = '../results/figures'
is_plot_title = False
is_save_fig = True

# print(plt.style.available)
# print(plt.rcParams)
plt.style.use(['seaborn-darkgrid', 'seaborn-paper'])
label_size = 18
tick_size = 14
plt.rc('text', usetex=False)
plt.rc('font', family='serif')
plt.rc('axes', titlesize=label_size)
plt.rc('axes', labelsize=label_size)
plt.rc('xtick', labelsize=tick_size)
plt.rc('ytick', labelsize=tick_size)



C:\Users\uria\vmplayer_shared\Pnml_code\deep_pnml\notebooks
Loading paths from result_summary.ipynb


# PNML Vs. ERM Performence
PNML is as training is as following:
1. Train base model with all adversarial trainset (Natural/FGSM/PGD)
2. Report the accuracy of the adversarial testset (Natural/FGSM/PGD) as ERM
Perform pNML procedure - 
3. For each test sample:
    1. For each label:
        1. Refine the sample to better fit the label using targeted FGSM attack
        2. Inference the refined sample and save the probability of the corresponding label.
    2. Take all save probabilities and normalize them.
    3. Report the new probability assignment

In [2]:
# Paper results:
# {Adv._train}_{Adv._attack}
###  MNIST: ###
mnist_results_d={}
# refinement : fgsm eps=0.1
# Natural training
mnist_results_d['natural_natural'] = ['./../results/paper/MNIST/mnist_adversarial_results_20190802_133952/results_mnist_adversarial_20190802_133952.json'] # [Original\PNML Accuracy] = [0.9934/0.9925]
mnist_results_d['natural_fgsm'] = ['./../results/paper/MNIST/mnist_adversarial_results_20190802_135239/results_mnist_adversarial_20190802_135239.json'] # [Original\PNML Accuracy] = [0.0056/0.0159]
mnist_results_d['natural_pgd'] = ['./../results/paper/MNIST/mnist_adversarial_results_20190802_131552/results_mnist_adversarial_20190802_131552.json'] # [Original\PNML Accuracy] = [0.0/0.0]
# FGSM training eps=0.3
mnist_results_d['fgsm_natural'] = ['./../results/paper/MNIST/mnist_adversarial_results_20190802_122620/results_mnist_adversarial_20190802_122620.json']
mnist_results_d['fgsm_fgsm'] = ['./../results/paper/MNIST/mnist_adversarial_results_20190801_192015/results_mnist_adversarial_20190801_192015.json'] # [Original\PNML Accuracy] = [0.976700\0.9135]
mnist_results_d['fgsm_pgd'] = ['./../results/paper/MNIST/mnist_adversarial_results_20190802_125138/results_mnist_adversarial_20190802_125138.json']# [Original\PNML Accuracy] = [0.0027/0.2329]
# PGD training eps=0.3,iter=40 , step_size=0.01, rnd_starts=20
mnist_results_d['pgd_natural'] = ['./../results/paper/MNIST/mnist_adversarial_results_20190802_153819/results_mnist_adversarial_20190802_153819.json']   # [Original\PNML Accuracy] = [0.9832\0.9831]
mnist_results_d['pgd_fgsm'] = ['./../results/paper/MNIST/mnist_adversarial_results_20190802_143802/results_mnist_adversarial_20190802_143802.json']  # [Original\PNML Accuracy] = [0.9577\0.9391]
mnist_results_d['pgd_pgd'] = ['./../results/paper/MNIST/mnist_adversarial_results_20190802_151544/results_mnist_adversarial_20190802_151544.json']  # [Original\PNML Accuracy] = [0.9051\0.9519]

###  CIFAR10: ###
cifar_results_d = {}
# Natural training
# refinement : fgsm eps=0.02
cifar_results_d['natural_natural'] = ['./../results/paper/CIFAR/table1/natural_train_lambda_002/natural/results_cifar_adversarial_20190923_140033.json']  # [Original\PNML Accuracy] = [] 
cifar_results_d['natural_fgsm'] = ['./../results/paper/CIFAR/table1/natural_train_lambda_002/fgsm/results_cifar_adversarial_20190923_135955.json']  # [Original\PNML Accuracy] = [] 
cifar_results_d['natural_pgd'] = ['./../results/paper/CIFAR/table1/natural_train_lambda_002/pgd/cifar_adversarial_results_20190923_140142/results_cifar_adversarial_20190923_140142.json']  # [Original\PNML Accuracy] = [] 
# FGSM training eps=0.0313
# refinement : fgsm eps=0.08
cifar_results_d['fgsm_natural'] = ['./../results/paper/CIFAR/table1/fgsm_train_lambda_008/cifar_fgsm_train_noattack/cifar_adversarial_results_20190924_221329/results_cifar_adversarial_20190924_221329.json']  # [Original\PNML Accuracy] = [] 
cifar_results_d['fgsm_fgsm'] = ['./../results/paper/CIFAR/table1/fgsm_train_lambda_008/cifar_adversarial_results_20190924_165015/results_cifar_adversarial_20190924_165015.json']  # [Original\PNML Accuracy] = [] 
cifar_results_d['fgsm_pgd'] = ['./../results/paper/CIFAR/table1/fgsm_train_lambda_008/cifar_fgsm_train_pgd_attack/cifar_adversarial_results_20190924_222226/results_cifar_adversarial_20190924_222226.json']  # [Original\PNML Accuracy] = []
# PGD training eps=0.0313,iter=20 , step_size=0.0078, rnd_starts=1
# refinement : fgsm eps=0.11
cifar_results_d['pgd_natural'] = ['./../results/paper/CIFAR/table1/cifar_adversarial_results_20190810_175715/results_cifar_adversarial_20190810_175715.json']  # [Original\PNML Accuracy] = [] 
cifar_results_d['pgd_fgsm'] = ['./../results/paper/CIFAR/table1/cifar_adversarial_results_20190810_193501/results_cifar_adversarial_20190810_193501.json']  # [Original\PNML Accuracy] = [] 
cifar_results_d['pgd_pgd'] = ['./../results/paper/CIFAR/table1/cifar_adversarial_results_20190810_134149/results_cifar_adversarial_20190810_134149.json']  # [Original\PNML Accuracy] = [0.3738\0.6086]

In [3]:
tic = time.time()
result_df, statistics_df = load_results_to_df(mnist_results_d['natural_natural'])
print('result_df: loaded in {0:.2f} [s]'.format(time.time() - tic))
print(statistics_df.transpose())
print(result_df.shape[0])

result_df: loaded in 2.21 [s]
          acc  mean loss  std loss  mean entropy
nml    0.9925   0.192152  0.347189      0.131640
erm    0.9934   0.025535  0.332086      0.004567
genie  0.9998   0.000327  0.016714      0.000266
10000


In [4]:
# Create Table 1 MNIST(from paper): Accuracy rate for various adversary attacks and adversary trained models 
for key in mnist_results_d:
    _, statistics_df = load_results_to_df(mnist_results_d[key])
    print("[Train_Attack]=[{}] Accuracy:[ERM / pNML]= [{} / {}]".format(key, statistics_df.loc['acc',"erm"],statistics_df.loc['acc',"nml"]))

[Train_Attack]=[natural_natural] Accuracy:[ERM / pNML]= [0.9934 / 0.9925]
[Train_Attack]=[natural_fgsm] Accuracy:[ERM / pNML]= [0.0056 / 0.0159]
[Train_Attack]=[natural_pgd] Accuracy:[ERM / pNML]= [0.0 / 0.0]
[Train_Attack]=[fgsm_natural] Accuracy:[ERM / pNML]= [0.971 / 0.9716]
[Train_Attack]=[fgsm_fgsm] Accuracy:[ERM / pNML]= [0.9767 / 0.9135]
[Train_Attack]=[fgsm_pgd] Accuracy:[ERM / pNML]= [0.0027 / 0.2329]
[Train_Attack]=[pgd_natural] Accuracy:[ERM / pNML]= [0.9832 / 0.9831]
[Train_Attack]=[pgd_fgsm] Accuracy:[ERM / pNML]= [0.9577 / 0.9391]
[Train_Attack]=[pgd_pgd] Accuracy:[ERM / pNML]= [0.9051 / 0.9519]


In [5]:
# Create Table 1 CIFAR(from paper): Accuracy rate for various adversary attacks and adversary trained models 
for key in cifar_results_d:
    _, statistics_df = load_results_to_df(cifar_results_d[key])
    print("[Train_Attack]=[{}] Accuracy:[ERM / pNML]= [{} / {}]".format(key, statistics_df.loc['acc',"erm"],statistics_df.loc['acc',"nml"]))

[Train_Attack]=[natural_natural] Accuracy:[ERM / pNML]= [0.9385 / 0.8725]
[Train_Attack]=[natural_fgsm] Accuracy:[ERM / pNML]= [0.0606 / 0.059]
[Train_Attack]=[natural_pgd] Accuracy:[ERM / pNML]= [0.0 / 0.0103]
[Train_Attack]=[fgsm_natural] Accuracy:[ERM / pNML]= [0.8347 / 0.7942]
[Train_Attack]=[fgsm_fgsm] Accuracy:[ERM / pNML]= [0.4944 / 0.5011]
[Train_Attack]=[fgsm_pgd] Accuracy:[ERM / pNML]= [0.3619 / 0.4882]
[Train_Attack]=[pgd_natural] Accuracy:[ERM / pNML]= [0.843 / 0.7905]
[Train_Attack]=[pgd_fgsm] Accuracy:[ERM / pNML]= [0.4788 / 0.5452]
[Train_Attack]=[pgd_pgd] Accuracy:[ERM / pNML]= [0.3738 / 0.6086]
